In [1]:
import tensorflow as tf
# import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data

tf.set_random_seed(777)

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

class Model:
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            
            self.X = tf.placeholder(tf.float32, [None, 28*28])
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32, kernel_size=[3,3],
                                    padding='SAME', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2],
                                           padding='SAME', strides=2)
            dropout1 = tf.layers.dropout(inputs=pool1,
                                        rate=0.7, training=self.training)
            
            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3],
                                    padding='SAME', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2],
                                     padding='SAME', strides=2)
            dropout2 = tf.layers.dropout(inputs=pool2,
                                        rate=0.7, training=self.training)
            
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3],
                                    padding='SAME', activation = tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2],
                                           padding='SAME', strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3,
                                        rate=0.7, training=self.training)
            
            flat = tf.reshape(dropout3, [-1, 128*4*4])
            dense4 = tf.layers.dense(inputs=flat,
                                    units=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4,
                                        rate=0.5, training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout4, units=10)
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.loss)
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, {self.X: x_test, self.training: training})
    
    def get_acc(self, x_test, y_test, training=False):
        return self.sess.run(self.acc, {self.X: x_test, self.Y: y_test, self.training: training})
    
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.optimizer, self.loss], {
            self.X: x_data, self.Y: y_data, self.training: training})

sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning started.')

for epoch in range(training_epochs):
    avg_loss = 0
    total_batch = int(mnist.train.num_examples/batch_size)
    
    for i in range(total_batch):
        x, y = mnist.train.next_batch(batch_size)
        _, l = m1.train(x, y)
        avg_loss += l/total_batch
        
    print('Epoch:', '%02d' % (epoch+1), 'loss = ', '{:.9f}'.format(avg_loss))
print('Learning ended.')
print('Accuracy:', m1.get_acc(mnist.test.images, mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Learning started.
Epoch: 01 loss =  0.812645804
Epoch: 02 loss =  0.294864882
Epoch: 03 loss =  0.227835996
Epoch: 04 loss =  0.197752685
Epoch: 05 loss =  0.181831977
Epoch: 06 loss =  0.172857242
Epoch: 07 loss =  0.163468029
Epoch: 08 loss =  0.152047243
Epoch: 09 loss =  0.149585148
Epoch: 10 loss =  0.149794981
Epoch: 11 loss =  0.141487017
Epoch: 12 loss =  0.138499418
Epoch: 13 loss =  0.134192500
Epoch: 14 loss =  0.139926521
Epoch: 15 loss =  0.133264621
Learning ended.
Accuracy: 0.9895
